Using Julia and our own package

In [ ]:
using StatsPlots
using Flux
using ParameterSchedulers
using SchedulingRL

# set up problem
Env = createenv(as = "AIM", instanceName = "lhmhlh-0_ii_Expo5")

# setup test instances
testenvs = [Dict("layout" => "Flow_shops" ,
                    "instancetype" => "benchmark" ,
                    "instancesettings" => "base_setting",
                    "datatype" => "data_ii",
                    "instancename" =>"lhmhlh-0_ii_Expo5")]

Training with approximator: E2E

In [3]:
AgentE2E = createagent(
    createE2E( 
            ActorCritic(
                actor = Chain(Dense(statesize(Env) => 300, relu),
                        Dense(300 => 300, relu), Dense(300, 300, relu), Dense(300 => actionsize(Env))), # no specified activation function = linear activation
                critic = Chain(Dense(statesize(Env) => 300, relu),
                        Dense(300 => 300, relu),Dense(300 => 150, relu), Dense(150, 60, relu), Dense(60,1)),
                optimizer = Adam(0.0001)),            # no specified activation function = linear activation
                [Env],
                [],
                #lr_scheduler = ParameterSchedulers.Stateful(Inv(λ = 1e-1, γ = 0.2, p = 2)),
            critic_loss_weight = 0.05f0,
            n_steps=256
            )
        , "AIM")


println("start E2E training")
trainagent!(AgentE2E, generations = 200, showprogressbar = false, TBlog = true)

Store results E2E

In [ ]:
resultsE2E = testagent(AgentE2E,testenvs,100)
    
println("The average PI gap is: ", resultsE2E[6])
println("The worst gap is: ", resultsE2E[8])
println("All gaps are: \n", resultsE2E[10])

Training with approximator: GP

In [ ]:
# train GP
AgentGP = createagent(createGP([Env]), "AIM")
trainagent!(AgentGP,generations = 4)

Store the results

In [ ]:
resultsGP = testagent(AgentGP,testenvs,100)
    
println("The average PI gap is: ", resultsGP[6])
println("The worst gap is: ", resultsGP[8])
println("All gaps are: \n", resultsGP[10])

Training with approximator: WPF

In [ ]:
model = Chain(Dense(statesize(Env) => 30, selu),
                Dense(30 => 30, selu),)

prio = Priorityfunction("PT + DD + RO + RW + ON + JS + RF + CT + EW + JWM + ST + NI + NW + SLA + CW + CJW + TT")
# prio = Priorityfunction("PT + ON") to test if arbitrary functions work


output = numberweights(prio)
AgentWPF = createagent(
    createWPF( 
            ActorCritic(
                actor = GaussianNetwork(
                    pre = Chain(model,Dense(30 => 30)),
                    μ = Chain(Dense(30, output, tanh)),
                    σ = Chain(Dense(30, output, sigmoid)),
                    ),
                critic = Chain(model,Dense(30 => 1))),
                [Env],
                [],
                prio,
                masked = false,
                lr_scheduler = ParameterSchedulers.Stateful(Inv(λ = 1e-1, γ = 0.2, p = 2))
            )
        , "AIM")

trainagent!(AgentWPF,generations = 100)

Store results WPF

In [ ]:
resultsWPF = testagent(AgentWPF,testenvs,100)
    println("The average PI gap is: ", resultsWPF[6])
    println("The worst gap is: ", resultsWPF[8])
    println("All gaps are: \n", resultsWPF[10])

Visualize Results

In [ ]:
using StatsPlots

# boxplot with gaps of all 3 models
boxplot([resultsGP[10],resultsE2E[10],resultsWPF[10]], 
    label = ["GP" "E2E" "WPF"],
    title = "Boxplot of gaps",
    ylabel = "gap",
    xlabel = "model")